In [1]:
import tensorflow as tf
import os
import gzip
import numpy as np
import argparse
from tensorflow.python.keras.callbacks import Callback

class MyFashionMnist(object):

  def train(self):
    
    # 학습수행결과 모델을 save_model
    parser = argparse.ArgumentParser()
    parser.add_argument('--learning_rate', required=False, type=float, default=0.001)
    parser.add_argument('--dropout_rate', required=False, type=float, default=0.3)
    parser.add_argument('--epoch', required=False, type=int, default=5)    
    parser.add_argument('--act', required=False, type=str, default='relu')        
    parser.add_argument('--layer', required=False, type=int, default=1)      
    parser.add_argument('--model_version', required=False, type=str, default='0001')    
    parser.add_argument('--checkpoint_dir', required=False, default='/reuslt/training_checkpoints')
    parser.add_argument('--saved_model_dir', required=False, default='/result/saved_model')        
    parser.add_argument('--tensorboard_log', required=False, default='/result/log')     
    args = parser.parse_args()    
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    
    for i in range(int(args.layer)):    
        model.add(tf.keras.layers.Dense(128, activation=args.act))
        if(i > 2) :
            model.add(tf.keras.layers.Dropout(args.dropout_rate))
        
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=args.learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
    
    model.summary()    
    
    # 체크포인트를 저장할 체크포인트 디렉터리를 지정합니다.
    checkpoint_dir = args.checkpoint_dir
    # 체크포인트 파일의 이름
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")        

    model.fit(x_train, y_train,
              verbose=0,
              validation_data=(x_test, y_test),
              epochs=args.epoch,
              callbacks=[KatibMetricLog(),
                        tf.keras.callbacks.TensorBoard(log_dir=args.tensorboard_log),
                        tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                               save_weights_only=True)
                        ])
    
    path = args.saved_model_dir + "/" + args.model_version        
    model.save(path, save_format='tf')

class KatibMetricLog(Callback):
    def on_batch_end(self, batch, logs={}):
        print("batch=" + str(batch),
              "accuracy=" + str(logs.get('acc')),
              "loss=" + str(logs.get('loss')))
    def on_epoch_begin(self, epoch, logs={}):
        print("epoch " + str(epoch) + ":")
    
    def on_epoch_end(self, epoch, logs={}):
        print("Validation-accuracy=" + str(logs.get('val_acc')),
              "Validation-loss=" + str(logs.get('val_loss')))
        return

if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils

        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        fairing.config.set_builder(
            'append',
            image_name='serving-job',
            base_image='brightfly/kubeflow-jupyter-lab:tf2.0-cpu', # here
            registry=DOCKER_REGISTRY, 
            push=True)
        # cpu 1, memory 5GiB
        fairing.config.set_deployer('job',
                                    namespace='admin',  # here namespace
                                    pod_spec_mutators=[
                                        k8s_utils.mounting_pvc(pvc_name="fashion-mnist", 
                                                              pvc_mount_path="/result"),
                                        k8s_utils.get_resource_mutator(cpu=1,
                                                                       memory=5)]
                                   )
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[I 201019 06:38:41 config:123] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7fec00677da0>
[I 201019 06:38:41 config:125] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7febb7d0a470>
[I 201019 06:38:41 config:127] Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7febb7d0a470>
[W 201019 06:38:41 append:50] Building image using Append builder...
[I 201019 06:38:41 base:105] Creating docker context: /tmp/fairing_context_uvmykuqg
[I 201019 06:38:41 converted_notebook:127] Converting 10-fmnist-save-model-renew.ipynb to 10-fmnist-save-model-renew.py
[I 201019 06:38:41 docker_creds_:234] Loading Docker credentials for repository 'brightfly/kubeflow-jupyter-lab:tf2.0-cpu'
[W 201019 06:38:44 append:54] Image successfully built in 2.426172080995457s.
[W 201019 06:38:44 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/serving-job:57B53938.

4423680/4422102 [==============================] - 0s 0us/step
2020-10-19 06:39:02.728942: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-10-19 06:39:02.749895: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000140000 Hz
2020-10-19 06:39:02.750549: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5f1b530 executing computations on platform Host. Devices:
2020-10-19 06:39:02.750589: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
2020-10-19 06:39:04.069024: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
flatten (Flatten)            (None, 784)               0
_____________________________________________

[W 201019 06:40:18 job:162] Cleaning up job fairing-job-nwp86...
